# 05_funcGLM2

这是功能像的1st GLM。

In [2]:
import os
import scipy.io as scio
from RZutilpy.system import Path, unix_wrapper, gettimestr, makedirs
from RZutilpy.rzio import matchfiles

ModuleNotFoundError: No module named 'RZutilpy'

# Step 1. 

复制GLM1的文件到temp文件夹

In [1]:
Group = scio.loadmat('/home/yufengxia/Data/23BPhappy/temp/clidata.mat')
Group = Group['clidata']



GroupMDD = []
GroupBD = []
#del folders[0:12] 

Param1 = 'option_CRandEV'
Param2 = 'gamble_CRandEV'
for i in range(0,len(Group)):
    subj = Group[i][0][0]
    folders = matchfiles(f'/home/yufengxia/Data/23BPhappy/*_BPhappy_{subj}')
    GLM_dir = Path(f'{folders}/funcGLM1')
    GLM_dir = Path(GLM_dir)
    if GLM_dir.exists() == False:
        continue

    if Group[i][10][0][0] == 0:
        GroupMDD.append(subj)
    elif Group[i][10][0][0] == 1:
            GroupBD.append(subj)

    statsfiles = matchfiles(f'{GLM_dir}/*{Param1}_{Param2}_stats.nii*')
    !cp {statsfiles}  /home/yufengxia/Data/23BPhappy/temp
    print(statsfiles)
    


NameError: name 'matchfiles' is not defined

In [10]:
Param1 = 'option_CRandEV'
Param2 = 'gamble_RPE'

setA = '-setA' 
setB = '-setB'
statsfiles = matchfiles(f'/home/yufengxia/Data/23BPhappy/temp/*{Param1}_{Param2}_stats.nii*')
for files in statsfiles:
    subj = files[36:40]
    if subj in GroupMDD:
        setA += ' ' + files + '\'[10]\''
    if subj in GroupBD:
        setB += ' ' + files + '\'[10]\''

print(setA)
print(setB)
%cd /home/yufengxia/Data/23BPhappy/temp
! 3dttest++ -prefix /home/yufengxia/Data/23BPhappy/results/GLM2_automask_{Param1}_{Param2}.dset {setA} {setB} -labelA MDD -labelB BD

['/home/yufengxia/Data/23BPhappy/temp/M002_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M003_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M005_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M006_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M007_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M008_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M009_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M010_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M011_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M012_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M013_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M014_option_hp_gamble_hp_stats.nii.gz', '/home/yufengxia/Data/23BPhappy/temp/M015_option_hp_gamble_hp_stats.nii.gz'

In [ ]:
#可视化结果
AFNI_HOME = os.getenv('AFNIDIR')
MNIT1 = AFNI_HOME + '/MNI152_2009_template_SSW.nii.gz'
%cd /home/yufengxia/Data/23BPhappy/results/
! afni -dset GLM2_{Param1}_{Param2}.dset+tlrc.BRIK  {MNIT1}